reference

* https://www.kaggle.com/kneroma/m5-first-public-notebook-under-0-50
* https://www.kaggle.com/kneroma/m5-forecast-v2-python

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.options.display.max_columns = 50
import gc
import lightgbm as lgb

In [ ]:
CAL_DTYPES = {  "event_name_1": "category", "event_type_1": "category", 
                "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int8",
              "month": "int8", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
STE_DTYPES = {  "item_id": "category", "dept_id": "category", "cat_id": "category", "store_id": "category",
              "state_id": "category"  }
PRICE_DTYPES = {  "store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [ ]:
cal = pd.read_csv("../input/m5-forecasting-accuracy/calendar.csv", dtype = CAL_DTYPES)
#stv = pd.read_csv("../input/m5-forecasting-accuracy/sales_train_validation.csv")
ste = pd.read_csv("../input/m5-forecasting-accuracy/sales_train_evaluation.csv", dtype = STE_DTYPES)
pri = pd.read_csv("../input/m5-forecasting-accuracy/sell_prices.csv", dtype = PRICE_DTYPES)

祝祭日特徴量の調整：chanukah8日, pesach7日に拡張 ramadan,NBAはそのまま

スコア変わらなかったのでやめる

In [ ]:
"""
days_chanukah = [333, 687, 1041, 1425, 1780]
for day in days_chanukah :
    for i in range(1,8) : 
        if pd.isnull(cal.loc[day-i,"event_name_1"]):
            cal.loc[day-i,"event_name_1"] = 'Chanukah End'
            cal.loc[day-i,"event_type_1"] = 'Religious'
        elif pd.isnull(cal.loc[day-i,"event_name_2"]):
            cal.loc[day-i,"event_name_2"] = 'Chanukah End'
            cal.loc[day-i,"event_type_2"] = 'Religious'
        else:
            print("canceled")

days_pesach = [87, 441, 794, 1179, 1533, 1918]
for day in days_pesach :
    for i in range(1,7) : 
        if pd.isnull(cal.loc[day-i,"event_name_1"]):
            cal.loc[day-i,"event_name_1"] = 'Pesach End'
            cal.loc[day-i,"event_type_1"] = 'Religious'
        elif pd.isnull(cal.loc[day-i,"event_name_2"]):
            cal.loc[day-i,"event_name_2"] = 'Pesach End'
            cal.loc[day-i,"event_type_2"] = 'Religious'
        else:
            print("canceled")
"""
print("canceled")

メモリを食うので型変換してから整形

In [ ]:
trans_cols = ["item_id", "dept_id", "cat_id", "store_id", "state_id"]
for col in trans_cols:
    ste[col] = ste[col].cat.codes.astype("int16")
trans_cols = ["item_id", "store_id"]
for col in trans_cols:
    pri[col] = pri[col].cat.codes.astype("int16")
trans_cols = ["event_name_1", "event_type_1"]
for col in trans_cols:
    cal[col] = cal[col].cat.codes.astype("int8") + 1
d_order = { 'Chanukah End': 0, 'Cinco De Mayo': 2, 'Easter': 4, "Father's day": 7, 'OrthodoxEaster': 20, "Purim End": 23  }
cal["event_name_2"] = cal["event_name_2"].map(d_order).fillna(-1).astype("int8") + 1
d_order = {  'Cultural': 0, 'Religious': 2  }
cal["event_type_2"] = cal["event_type_2"].map(d_order).fillna(-1).astype("int8") + 1
cal["date"] = pd.to_datetime(cal["date"])

In [ ]:
cal.drop(['weekday'], axis=1, inplace=True)

In [ ]:
df = pd.melt(ste,id_vars=ste.columns.values[:6],var_name="d",value_name="sells")
df["sells"] = df["sells"].astype("float32")

df = df.merge(cal, on='d', copy = False)
df = df.merge(pri, on=["store_id", "item_id", "wm_yr_wk"],copy = False)
# df = df.sort_values(["wm_yr_wk", "wday"]) # memory over

やばそうなデータを削る EDA参照

→結局メモリの関係で最近1年半のデータに絞った

In [ ]:
df.drop(df.index[df["wm_yr_wk"]<=11430], inplace=True)
days_christmas = ["d_331","d_697","d_1062","d_1427","d_1792"]
for day in days_christmas:
    df.drop(df.index[df["d"] == day], inplace=True)

In [ ]:
# df.drop(df.index[df["wm_yr_wk"]<=11200], inplace=True)
# df.drop(df.index[(df["wm_yr_wk"]<=11240) & (df["cat_id"]==0)], inplace=True)
# df.drop(df.index[(df["wm_yr_wk"]<=11248) & (df["store_id"]==7)], inplace=True)
# df.drop(df.index[(df["wm_yr_wk"]<=11230) & (df["store_id"]==8)], inplace=True)

特徴量の追加　週月前と移動平均、max

rollingが処理をを食うので工夫が必要？なんでだろう

* 価格の移動平均もつけてみる

In [ ]:
def create_features(dt):
    dt["lag_7"] = dt[["id","sells"]].groupby("id")["sells"].shift(7)
    dt["lag_28"] = dt[["id","sells"]].groupby("id")["sells"].shift(28)

    dt["win_7"] = dt[["id","sells"]].groupby("id")["sells"].transform(lambda x : x.rolling(7).mean()).shift(1)
    dt["win_28"] = dt[["id","sells"]].groupby("id")["sells"].transform(lambda x : x.rolling(28).mean()).shift(1)

    dt["win_7_lag_7"] = dt[["id","win_7"]].groupby("id")["win_7"].shift(7)
    dt["win_28_lag_28"] = dt[["id","win_28"]].groupby("id")["win_28"].shift(28)
    
    # dt["max_7"] = dt[["id","sells"]].groupby("id")["sells"].transform(lambda x : x.rolling(7).max()).shift(1)
    # dt["max_28"] = dt[["id","sells"]].groupby("id")["sells"].transform(lambda x : x.rolling(28).max()).shift(1)
    
    dt["price_lag_1"] = dt[["id","sell_price"]].groupby("id")["sell_price"].shift(1)
    dt["price_lag_7"] = dt[["id","sell_price"]].groupby("id")["sell_price"].shift(7)
    
    dt["price_win_7"] = dt[["id","sell_price"]].groupby("id")["sell_price"].transform(lambda x : x.rolling(7).mean()).shift(1)
    dt["price_win_28"] = dt[["id","sell_price"]].groupby("id")["sell_price"].transform(lambda x : x.rolling(28).mean()).shift(1)

In [ ]:
create_features(df)
df.dropna(inplace = True)

trainデータに整形　カテゴリはlightgbmでなんとかしてくれる

In [ ]:
features =['item_id', 'dept_id', 'store_id', 'cat_id', 'state_id',
            'wday', 'month', 'year', 
            'event_name_1', 'event_name_2', 'event_type_1', 'event_type_2',
            'snap_CA', 'snap_TX', 'snap_WI',
            'sell_price',
            'lag_7', 'lag_28', 'win_7', 'win_28', 'win_7_lag_7','win_28_lag_28',
            'price_win_7', 'price_win_28']
X_train = df[features]
y_train = df['sells']
cat_features = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id', 
                'event_name_1', 'event_name_2', 'event_type_1', 'event_type_2']

In [ ]:
np.random.seed(1000)

fake_valid_index = np.random.choice(X_train.index.values, 2000000, replace = False)
train_index = np.setdiff1d(X_train.index.values, fake_valid_index)
train_data = lgb.Dataset(X_train.loc[train_index] , label = y_train.loc[train_index], 
                         categorical_feature=cat_features, free_raw_data=False)
fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_index], label = y_train.loc[fake_valid_index],
                              categorical_feature=cat_features, free_raw_data=False)

In [ ]:
del df, X_train, y_train, fake_valid_index, train_index
gc.collect()

これを見つつパラメータ設定

https://lightgbm.readthedocs.io/en/latest/Parameters.html

In [ ]:
params = {
    "objective" : "tweedie", #"poisson",
    # "alpha" : 0.9,
    "metric" : ["rmse"],
    "force_row_wise" : True,
    "learning_rate" : 0.07,
    "bagging_freq" : 3,
    "bagging_fraction" : 0.5,
    "lambda_l2" : 0.1,
    "num_iterations" : 1000,
    "num_leaves" : 255,
    "min_data_in_leaf": 128,
}

In [ ]:
%%time
m_lgb = lgb.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=50)

In [ ]:
m_lgb.save_model("model.lgb")

prediction用データの形成

基本同じ事をやって、日付毎に推定していく

56日前から必要

In [ ]:
days = [f"d_{i}" for i in range(1942,1970)]
for day in days:
    ste[day] = 0

In [ ]:
df = pd.melt(ste,id_vars=ste.columns.values[:6],var_name="d",value_name="sells")
df["sells"] = df["sells"].astype("float32")

df = df.merge(cal, on='d', copy = False)
df = df.merge(pri, on=["store_id", "item_id", "wm_yr_wk"],copy = False)

In [ ]:
df.drop(df.index[df["wm_yr_wk"]<=11607], inplace=True)

In [ ]:
create_features(df)

In [ ]:
%%time

for day in days:
    X_pred = df[df["d"] == day][features]
    y_pred = m_lgb.predict(X_pred)
    print(day)
    df.loc[df["d"] == day, "sells"] = y_pred
    create_features(df)

提出用データの作成　public, private双方が必要

In [ ]:
sub = df[["id","d","sells"]].pivot(index="id", columns="d", values="sells")
sub = sub.reset_index()
sub.columns.name = None

In [ ]:
sub1 = pd.concat([sub.T[0:1],sub.T[-56:-28]]).T
sub2 = pd.concat([sub.T[0:1],sub.T[-28:]]).T

In [ ]:
sub_columns = ["id"] + [f"F{i}" for i in range(1,29)]
sub1.columns = sub_columns
sub2.columns = sub_columns
sub1["id"] = sub1["id"].str.replace("evaluation", "validation")

In [ ]:
sub = pd.concat([sub2,sub1])
sub.to_csv("submission.csv",index=False)
sub